In [ ]:
# Packages
list.of.packages <- c("rstudioapi", "dplyr", "astsa", "zoo", "forecast", "ggplot2", "plotly", "tseries", 
                      "lubridate", "TSstudio", "sweep", "timetk", "tidyverse")
new.packages <- list.of.packages[!(list.of.packages %in% installed.packages()[,"Package"])]
if(length(new.packages)) install.packages(new.packages)

In [ ]:
require("rstudioapi")
require("dplyr")
require("astsa")
require("zoo")
require("forecast")
require("ggplot2")
require("plotly")
require("tseries")
require("lubridate")
require("TSstudio")
require("xts")
require("sweep")
require("timetk")
require("tidyverse")

In [ ]:
#Load folders
#lpd <- file.path("../Data/Lastprofildaten")
#wasser <- file.path("../Data/Wasserdaten")
#wetter <- file.path("../Data/Wetterdaten")
output <- file.path("../Data/Outputdaten")
forecast <- file.path("../Data/Forecast")

#Load dataframes from pickle
mm_all <- read.csv2(file.path(output, "mm_all.csv"))
pm_all <- read.csv2(file.path(output, "pm_all.csv"))
tm_all <- read.csv2(file.path(output, "tm_all.csv"))

In [ ]:
# Load dataframes only kw data and make type transformations
mm_kw <- mm_all[, c('timestamp', 'kw')]
pm_kw <- pm_all[, c('timestamp', 'kw')]
tm_kw <- tm_all[, c('timestamp', 'kw')]

mm_kw$kw <- as.numeric(mm_kw$kw)
pm_kw$kw <- as.numeric(pm_kw$kw)
tm_kw$kw <- as.numeric(tm_kw$kw)

mm_kw$timestamp <- as.POSIXct(mm_kw$timestamp, format = '%Y-%m-%d %H:%M:%S')
pm_kw$timestamp <- as.POSIXct(pm_kw$timestamp, format = '%Y-%m-%d %H:%M:%S')
tm_kw$timestamp <- as.POSIXct(tm_kw$timestamp, format = '%Y-%m-%d %H:%M:%S')

In [ ]:
# Check Null Values
mm_kw[is.na(mm_kw$kw),]
pm_kw[is.na(pm_kw$kw),]
tm_kw[is.na(tm_kw$kw),]

In [ ]:
#Aufgrund der Zeitumstellung gibt es NA-Values -> werden aus dem Dataframe entfernt
mm_kw <- mm_kw[!is.na(mm_kw$kw), ]
pm_kw <- pm_kw[!is.na(pm_kw$kw), ]
tm_kw <- tm_kw[!is.na(tm_kw$kw), ]

In [ ]:
# Change index
rownames(mm_kw) <- mm_kw$timestamp
rownames(pm_kw) <- pm_kw$timestamp
rownames(tm_kw) <- tm_kw$timestamp

In [ ]:
# Visualise Timeseries
tm_p1 <- tm_kw %>%
  ggplot(aes(x = timestamp, y = kw)) +
  geom_line(color = "#0099FF", size=0.1) + 
  scale_y_continuous() +
  labs(title = "Teufelmühle kw", y = "kw", x = "date") 

embed_notebook(ggplotly(tm_p1))

In [ ]:
## Reframing Timeseries
#mm_kw <- mm_kw %>% filter(timestamp >= "2010-07-01 00:00:00")
#pm_kw <- pm_kw %>% filter(timestamp >= "2019-03-01 00:00:00")
#tm_kw <- tm_kw %>% filter(timestamp >= "2019-03-01 00:00:00")

time_index <- seq(from = as.POSIXct("2013-01-01 09:00"), 
                  to = as.POSIXct("2021-08-31 23:00"), by = "hour")
tm_kw$timestamp <- time_index
rownames(tm_kw) <- tm_kw$timestamp

In [ ]:
# Generate Daily Data
tm_kw_d <- tm_kw %>% mutate(timestamp = floor_date(timestamp, unit = "day")) %>% 
  group_by(timestamp) %>% summarise(kw = sum(kw))

In [ ]:
a <- tm_kw_d %>% 
  plot_time_series(timestamp, kw,
                   .interactive = TRUE,
                   .plotly_slider = TRUE)
embed_notebook(a)

In [ ]:
# Generate Timeseries object

#hourly tk_ts
ts_h_train <- tm_kw %>%
    filter(timestamp < "2021-08-01") 

ts_h_test <- tm_kw %>%
    filter(timestamp >= "2021-08-01")


ts_h <- tk_ts(tm_kw, freq=24, start= c(2013, 1,1))
ts_h_train <- tk_ts(ts_h_train, freq=24, start = c(2013, 1,1))
ts_h_test <- tk_ts(ts_h_test, freq=24)

In [ ]:
#daily tk_ts
ts_d_train <- tm_kw_d %>%
    filter(timestamp < "2021-08-01") 

ts_d_test <- tm_kw_d %>%
    filter(timestamp >= "2021-08-01")

ts_d <- tk_ts(tm_kw_d, freq=365, start = c(2013, 1,1))
ts_d_train <- tk_ts(ts_d_train, freq=365, start = c(2013, 1,1))
ts_d_test <- tk_ts(ts_d_test, freq=365)

In [ ]:
b <- tm_kw_d %>%
    plot_stl_diagnostics(timestamp, kw, 
                         .frequency = "auto", .trend = "auto",
                         .interactive = TRUE)
embed_notebook(b)

In [ ]:
c <- tm_kw_d %>%
    plot_seasonal_diagnostics(timestamp, kw, .interactive = TRUE)
embed_notebook(c)

In [ ]:
tk_index(ts_h_test, timetk_idx = TRUE) 

In [ ]:
# Autocorrelation
acf2(ts_d)
acf2(ts_d, max.lag = 730)

In [ ]:
# stationary test
adf.test(ts_d)
adf.test(ts_h)

In [ ]:
#Fit Arima
fit_arima_1 <- auto.arima(ts_d_train, seasonal = TRUE, trace = TRUE)
print(summary(fit_arima_1))
checkresiduals(fit_arima_1)
#autoplot(fit_arima_1)


fcast_1 <- forecast(fit_arima_1, h = 31)
autoplot(fcast_1)
par(new=TRUE)
plot(ts_d_train)
print(summary(fcast_1))
print(fcast_1)

#accuracy(fcast, ts_d_test)

In [ ]:
sw_sweep(fcast_1) %>%
    ggplot(aes(x = index, y = kw, color = key)) +
    geom_ribbon(aes(ymin = lo.95, ymax = hi.95), 
                fill = "#D5DBFF", color = NA, size = 0) +
    geom_ribbon(aes(ymin = lo.80, ymax = hi.80, fill = key), 
                fill = "#596DD5", color = NA, size = 0, alpha = 0.8) +
    geom_line(size = 1) +
    labs(title = "Teufelmühle Daily Forecast", x = "datetime", y = "kwH") +
    scale_y_continuous()

In [ ]:
sw_sweep(fcast_1, timetk_idx = TRUE) %>%
    tail(40)

In [ ]:
fc1 <- sw_sweep(fcast_1, timetk_idx = TRUE) %>% plyr::rename(c("kw" = "prediction_sarima")) %>% filter(key == "forecast") %>% select(-key) 
tm_arima_day <- left_join(tm_kw_d, fc1,by = c("timestamp" = "index"))
write.csv(tm_arima_day,file.path(forecast, "tm_arima_d_1.csv"), row.names=FALSE)

In [ ]:
#Fit Arima force Seasonal
fit_arima_2 <- auto.arima(ts_d_train, D=1, trace = TRUE)
print(summary(fit_arima_2))
checkresiduals(fit_arima_2)

fcast_2 <- forecast(fit_arima_2, h = 31)
autoplot(fcast_2)
par(new=TRUE)
plot(ts_d)
print(summary(fcast_2))
print(fcast_2)

In [ ]:
sw_sweep(fcast_2) %>%
    ggplot(aes(x = index, y = kw, color = key)) +
    geom_ribbon(aes(ymin = lo.95, ymax = hi.95), 
                fill = "#D5DBFF", color = NA, size = 0) +
    geom_ribbon(aes(ymin = lo.80, ymax = hi.80, fill = key), 
                fill = "#596DD5", color = NA, size = 0, alpha = 0.8) +
    geom_line(size = 1) +
    labs(title = "Teufelmühle Daily Forecast", x = "datetime", y = "kwH") +
    scale_y_continuous()

In [ ]:
sw_sweep(fcast_2, timetk_idx = TRUE) %>%
    tail(40)

In [ ]:
fc2 <- sw_sweep(fcast_2, timetk_idx = TRUE) %>% plyr::rename(c("kw" = "prediction_sarima")) %>% filter(key == "forecast") %>% select(-key) 
tm_arima_day <- left_join(tm_kw_d, fc2,by = c("timestamp" = "index"))
write.csv(tm_arima_day,file.path(forecast, "tm_arima_d_2.csv"), row.names=FALSE)

In [ ]:
# hourly
fit_arima_3 <- auto.arima(ts_h_train, seasonal = TRUE, trace = TRUE)
print(summary(fit_arima_3))
checkresiduals(fit_arima_3)

fcast_3 <- forecast(fit_arima_3, h = 744)
autoplot(fcast_3)
par(new=TRUE)
plot(ts_h)
print(summary(fcast_3))
print(fcast_3)

In [ ]:
sw_sweep(fcast_3) %>%
    ggplot(aes(x = index, y = kw, color = key)) +
    geom_ribbon(aes(ymin = lo.95, ymax = hi.95), 
                fill = "#D5DBFF", color = NA, size = 0) +
    geom_ribbon(aes(ymin = lo.80, ymax = hi.80, fill = key), 
                fill = "#596DD5", color = NA, size = 0, alpha = 0.8) +
    geom_line(size = 1) +
    labs(title = "Teufelmühle Daily Forecast", x = "datetime", y = "kwH") +
    scale_y_continuous()

In [ ]:
sw_sweep(fcast_3, timetk_idx = TRUE) %>%
    tail(800)

In [ ]:
fc3 <- sw_sweep(fcast_3, timetk_idx = TRUE) %>% plyr::rename(c("kw" = "prediction_sarima")) %>% filter(key == "forecast") %>% select(-key) 
tm_arima_hour <- left_join(tm_kw, fc3,by = c("timestamp" = "index"))
write.csv(tm_arima_hour,file.path(forecast, "tm_arima_h_1.csv"), row.names=FALSE)

In [ ]:
# hourly
fit_arima_4 <- auto.arima(ts_h_train, D = TRUE, trace = TRUE)
print(summary(fit_arima_3))
checkresiduals(fit_arima_3)

fcast_4 <- forecast(fit_arima_4, h = 744)
autoplot(fcast_4)
par(new=TRUE)
plot(ts_h)
print(summary(fcast_4))
print(fcast_4)

In [ ]:
sw_sweep(fcast_4) %>%
    ggplot(aes(x = index, y = kw, color = key)) +
    geom_ribbon(aes(ymin = lo.95, ymax = hi.95), 
                fill = "#D5DBFF", color = NA, size = 0) +
    geom_ribbon(aes(ymin = lo.80, ymax = hi.80, fill = key), 
                fill = "#596DD5", color = NA, size = 0, alpha = 0.8) +
    geom_line(size = 1) +
    labs(title = "Teufelmühle Daily Forecast", x = "datetime", y = "kwH") +
    scale_y_continuous()

In [ ]:
sw_sweep(fcast_4, timetk_idx = TRUE) %>%
    tail(800)

In [ ]:
fc4 <- sw_sweep(fcast_4, timetk_idx = TRUE) %>% plyr::rename(c("kw" = "prediction_sarima")) %>% filter(key == "forecast") %>% select(-key) 
tm_arima_hour <- left_join(tm_kw, fc4,by = c("timestamp" = "index"))
write.csv(tm_arima_hour,file.path(forecast, "tm_arima_h_2.csv"), row.names=FALSE)